In [1]:
import sys
sys.path.append("/home/xucong24")
sys.path.append("/home/xucong24/ModernBERT")

import os
import math
import torch
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from Compiler.src.utils.system import read_ir_from_file
from sklearn.model_selection import StratifiedKFold
from Compiler.src.observation.inst2vec import Inst2vecEncoder
from Compiler.src.utils.llvm import compile_c_to_llvm_ir

In [2]:
from typing import cast
import src.mosaic_bert as mosaic_bert_module
from omegaconf import DictConfig
from omegaconf import OmegaConf as om
from transformers import AutoTokenizer

In [4]:
with open('/home/xucong24/ModernBERT/yamls/defaults.yaml') as f:
    default_cfg = om.load(f)

with open('/home/xucong24/ModernBERT/yamls/main/mosaic-bert-base-uncased.yaml') as f:
    yaml_cfg = om.load(f)

cfg = om.merge(default_cfg, yaml_cfg)
cfg = cast(DictConfig, cfg)  # for type checking
cfg.model['pretrained_checkpoint'] = '/home/xucong24/ModernBERT/mosaic-bert-base-uncased/ckpt/latest-rank0.pt'
cfg.model['pretrained_model_name'] = "/home/xucong24/bert-base-uncased"
cfg.model['tokenizer_name'] = "/home/xucong24/bert-base-uncased"


In [5]:
cfg.model

{'name': 'mosaic_bert', 'model_config': {'normalization': 'layernorm', 'hidden_act': 'gelu', 'num_attention_heads': 12, 'num_hidden_layers': 12, 'attention_probs_dropout_prob': 0.0, 'deterministic_fa2': False}, 'pretrained_model_name': '/home/xucong24/bert-base-uncased', 'tokenizer_name': '/home/xucong24/bert-base-uncased', 'pretrained_checkpoint': '/home/xucong24/ModernBERT/mosaic-bert-base-uncased/ckpt/latest-rank0.pt'}

In [6]:
print("Initializing model...")
modern_bert_model = mosaic_bert_module.create_mosaic_bert_mlm(
    pretrained_model_name=cfg.model.pretrained_model_name,
    pretrained_checkpoint=cfg.model.get("pretrained_checkpoint", None),
    model_config=cfg.model.get("model_config", None),
    tokenizer_name=cfg.model.get("tokenizer_name", None),
    gradient_checkpointing=cfg.model.get("gradient_checkpointing", None),
)
modern_bert_model = modern_bert_model.to(device='cuda')
n_params = sum(p.numel() for p in modern_bert_model.parameters())
print(f"{n_params=:.4e}")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Initializing model...


/home/xucong24/ModernBERT/src/bert_layers/attention.py:89: UserWarning: Unable to import flash_attn; defaulting MosaicBERT attention implementation to vanilla PyTorch (this will reduce throughput when using this model).
  warnings.warn(
/home/xucong24/ModernBERT/src/bert_layers/model.py:332: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_on

n_params=1.3740e+08


In [7]:
modern_bert_model

HuggingFaceModel(
  (model): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertAlibiEmbeddings(
        (word_embeddings): Embedding(30528, 768, padding_idx=0)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertAlibiEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertAlibiLayer(
            (attention): BertAlibiUnpadAttention(
              (self): BertAlibiUnpadSelfAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )


In [9]:
tokenizer = AutoTokenizer.from_pretrained("/home/xucong24/bert-base-uncased")
inputs = tokenizer("Hello, this is a [MASK] example.", return_tensors="pt").to(device='cuda')
outputs = modern_bert_model.model.bert(**inputs, output_hidden_states=True, return_dict=True)


In [10]:
outputs[0].shape

torch.Size([1, 10, 768])

In [11]:
outputs[0].squeeze().shape

torch.Size([10, 768])

In [12]:
modern_bert_model.eval()

HuggingFaceModel(
  (model): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertAlibiEmbeddings(
        (word_embeddings): Embedding(30528, 768, padding_idx=0)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertAlibiEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertAlibiLayer(
            (attention): BertAlibiUnpadAttention(
              (self): BertAlibiUnpadSelfAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )


In [13]:
data_folder = 'data/threadcoarsening_data'
platform = 'all'
num_epochs = 50
batch_size = 64
dense_layer_size = 32
print_summary = False
out_folder = 'output/modernbert_for_threadcoarsening'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cfs = [1, 2, 4, 8, 16, 32]  # thread coarsening factors
platform_list = ["Cypress", "Tahiti", "Fermi", "Kepler"]
_FLAG_TO_DEVICE_NAME = {
    'Cypress': 'AMD Radeon HD 5900',
    'Tahiti': 'AMD Tahiti 7970',
    'Fermi': 'NVIDIA GTX 480',
    'Kepler': 'NVIDIA Tesla K20c'
}

if not os.path.exists(out_folder):
    os.makedirs(out_folder)

assert platform in ["all", "Cypress", "Tahiti", "Fermi", "Kepler"], \
        'Choose device among: all, Cypress, Tahiti, Fermi, Kepler'

In [20]:
def find_runtime(df, kernel, cf, platform):
    filter1 = df["kernel"] == kernel
    filter2 = df["cf"] == cf
    return df.where(filter1 & filter2)["runtime_" + platform].dropna()


def load_data(data_folder):
    oracle_file = os.path.join(data_folder + "/pact-2014-oracles.csv")
    oracles = pd.read_csv(oracle_file)

    runtimes_file = os.path.join(data_folder + "/pact-2014-runtimes.csv")
    df = pd.read_csv(runtimes_file)

    cfs = np.array([1, 2, 4, 8, 16, 32])
    kernel_freq = df["kernel"].value_counts().sort_index().reset_index()

    inferencetime = []
    llfiles = pd.read_csv(data_folder + "/all.txt", sep="\s+")
    fileNum = llfiles["FileNum"]
    filesname = llfiles["ProgramName"]
    oracles["kernel_path"] = str("./") + oracles["kernel"] + str(".ll")
    resultant_data = pd.DataFrame()
    for i, platform in enumerate(platform_list):
        data = pd.merge(
            llfiles, oracles, left_on="ProgramName", right_on="kernel_path"
        )
        data["cf"] = data["cf_" + platform]
        data["device"] = i + 1
        resultant_data = pd.concat([resultant_data, data])

    resultant_data = pd.get_dummies(resultant_data, columns=["device"])
    resultant_data.reset_index(inplace=True)

    print('--- Preparing to read', len(resultant_data), 'input files from folder', data_folder + '/kernels_ir/')
    seqs = list()
    num_unks = 0

    # 遍历文件
    for file in tqdm(resultant_data["kernel"], desc='Encoding files'):
        # print(file)
        file = data_folder + '/kernels_ir/' + file + str(".ll")
        if os.path.exists(file):

            with open(file) as f:
                ir = f.read()

            with torch.no_grad():
                # 假设 data 是输入张量或字典
                inputs = tokenizer(ir, return_tensors="pt", max_length=512).to(device='cuda')

                outputs = modern_bert_model.model.bert(**inputs, output_hidden_states=True, return_dict=True)

                seqs.append(outputs[0].squeeze().to('cpu'))

        else:
            raise FileNotFoundError('Input file not found: ' + file)

    targetLabel = resultant_data["cf"]
    data = resultant_data
    data = data.drop(
        columns=[
            "index",
            "FileNum",
            "ProgramName",
            "kernel",
            "cf_Fermi",
            "runtime_Fermi",
            "cf_Kepler",
            "runtime_Kepler",
            "cf_Cypress",
            "runtime_Cypress",
            "cf_Tahiti",
            "runtime_Tahiti",
            "kernel_path",
            "cf",
        ]
    )

    return seqs, data, targetLabel, df, oracles

class ThreadCoaDataset(Dataset):
    def __init__(self, seqs, data, targetLabel):
        super().__init__()
        self.sequences = seqs
        self.dev = data
        self.y = targetLabel
        
    def __getitem__(self, index):
        seqs = self.sequences[index]
        dev = self.dev[index]
        label = self.y[index]
        return seqs, dev, label

    def __len__(self):
        return len(self.y)

In [15]:
# 定义网络结构
class ThreadCoaLSTM(nn.Module):
    def __init__(self, embedding_dim, dense_layer_size):
        super(ThreadCoaLSTM, self).__init__()
        self.lstm_1 = nn.LSTM(embedding_dim, embedding_dim, batch_first=True)
        self.lstm_2 = nn.LSTM(embedding_dim, embedding_dim, batch_first=True)
        self.language_model_out = nn.Linear(embedding_dim, 6)
        self.batch_norm = nn.BatchNorm1d(embedding_dim + 4)
        self.dense_1 = nn.Linear(embedding_dim + 4, dense_layer_size)
        self.output = nn.Linear(dense_layer_size, 6)
        
    def forward(self, x, device_input):
        out, _ = self.lstm_1(x)
        out, _ = self.lstm_2(out)
        lang_output = torch.sigmoid(self.language_model_out(out[:, -1, :]))
        x_combined = torch.cat((device_input, out[:, -1, :]), dim=1)
        x_combined = self.batch_norm(x_combined)
        x_combined = torch.relu(self.dense_1(x_combined))
        final_output = torch.sigmoid(self.output(x_combined))
        return final_output, lang_output


In [16]:
def eval_model(model, loader):
    model.eval()
    correct = 0
    pred_list, label_list =  [], []
    with torch.no_grad():
        for batch in loader:
            sequences, aux_input, labels = [b.to(device) for b in batch]
            outputs, lang_outputs = model(sequences, aux_input)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            pred_list.extend(preds.tolist())
            label_list.extend(batch[2].tolist())
            
    accuracy = correct / len(loader.dataset)
    return accuracy, pred_list, label_list

def train_model(model, train_loader, test_loader,  criterion, optimizer, num_epochs, model_path):
    # 模型训练
    pre_eval_acc = -1
    for epoch in range(num_epochs):
        epoch_loss = 0
        correct = 0
        model.train()
        for batch in train_loader:
            sequences, aux_input, labels = [b.to(device) for b in batch]
            
            optimizer.zero_grad()
            outputs, lang_outputs = model(sequences, aux_input)

            # 计算loss值 由output和lang_outputs与label计算CrossEntropyLoss
            loss = criterion(outputs, labels) + 0.2 * criterion(lang_outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
                    
        accuracy = correct / len(train_loader.dataset)
        eval_acc,_ , _ = eval_model(model, test_loader)
        # print(f"epoch {epoch+1}/{num_epochs}, loss: {epoch_loss:.4f}, train_acc: {accuracy:.4f}, eval_acc: {eval_acc:.4f}")

        if eval_acc > pre_eval_acc:
            pre_eval_acc = eval_acc
            torch.save(model.state_dict(), model_path)
    
    model.load_state_dict(torch.load(model_path, weights_only=False))


In [17]:
from sklearn.model_selection import KFold

In [21]:
# platform_list = ["amd", "nvidia"]
sequences, data, targetLabel, df, oracles = load_data(data_folder)
device_onehot_tensor = torch.tensor(data.values, dtype=torch.float32)
y_tensor = torch.argmax(torch.tensor(pd.get_dummies(targetLabel).values, dtype=torch.float32), dim=1)
cfs = np.array([1.0, 2.0, 4.0, 8.0, 16.0, 32.0])
kernel_freq = df["kernel"].value_counts().sort_index().reset_index()

--- Preparing to read 68 input files from folder data/threadcoarsening_data/kernels_ir/


Encoding files: 100%|██████████| 68/68 [00:02<00:00, 25.06it/s]


In [28]:
data = []
kf = KFold(n_splits=len(targetLabel), shuffle=False)
for j, (train_index, test_index) in enumerate(kf.split(sequences, targetLabel)):
    print('--- Cross validation step [', j, '/ ',len(targetLabel),' ]')
    kernel = sorted(set(df["kernel"]))[test_index[0] % 17]

    model_basename = 'lstm'
    model_path = os.path.join(out_folder, f"models/{model_basename}-{platform}-{j}.pth")
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    log_dir = os.path.join(out_folder, "logs")
    
    # 读取数据集
    train_data = ThreadCoaDataset([sequences[i] for i in train_index], device_onehot_tensor[train_index], y_tensor[train_index])
    test_data = ThreadCoaDataset([sequences[i] for i in test_index], device_onehot_tensor[test_index], y_tensor[test_index])
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    if not os.path.exists(model_path):
        # 创建模型
        model = ThreadCoaLSTM(embedding_dim=768, dense_layer_size=dense_layer_size).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        print('--- Training model... ')
        train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, model_path)
        
    else:
        # 读取模型权重文件
        model = ThreadCoaLSTM(embedding_dim=768, dense_layer_size=dense_layer_size).to(device)
        model.load_state_dict(torch.load(model_path, weights_only=False))
        model = model.to(device)
        print("Found trained model in", model_path, ", skipping...")

    # 模型预测
    accuracy, pred_list, label_list = eval_model(model, test_loader)
    # accuracy, pred_list, label_list
    prediction = cfs[pred_list[0]]

    if device_onehot_tensor[test_index].tolist()[0][0] == 1:
        platform = platform_list[0]
    elif device_onehot_tensor[test_index].tolist()[0][1] == 1:
        platform = platform_list[1]
    elif device_onehot_tensor[test_index].tolist()[0][2] == 1:
        platform = platform_list[2]
    elif device_onehot_tensor[test_index].tolist()[0][3] == 1:
        platform = platform_list[3]

    oracle_runtimes = np.array([float(x) for x in oracles["runtime_" + platform]])
    oracle = targetLabel[test_index[0]]
    print(oracle == prediction)

    rt_baseline = float(find_runtime(df, kernel, 1, platform))
    # rt_pred = float(find_runtime(df, kernel, prediction, platform))
    result = find_runtime(df, kernel, prediction, platform)
    if not result.empty:
        rt_pred = float(result.iloc[0])
    else:
        # rt_pred = float('nan')  # 或你可以 raise 或跳过这个样本
        print('error')
        continue
    rt_oracle = float(oracle_runtimes[test_index[0] % 17])
    data.append(
        {
            "Model": "IR2vec",
            "Platform": _FLAG_TO_DEVICE_NAME[platform],
            "Kernel": kernel,
            "Oracle-CF": oracle,
            "Predicted-CF": prediction,
            "Speedup": rt_baseline / rt_pred,
            "Oracle": rt_oracle / rt_pred,
            "OracleSpeedUp": rt_baseline / rt_oracle,
        }
    )

--- Cross validation step [ 0 /  68  ]
--- Training model... 
True
--- Cross validation step [ 1 /  68  ]


/tmp/ipykernel_59994/3411885580.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rt_baseline = float(find_runtime(df, kernel, 1, platform))


Found trained model in output/modernbert_for_threadcoarsening/models/lstm-Cypress-1.pth , skipping...
True
--- Cross validation step [ 2 /  68  ]


/tmp/ipykernel_59994/3411885580.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rt_baseline = float(find_runtime(df, kernel, 1, platform))


Found trained model in output/modernbert_for_threadcoarsening/models/lstm-Cypress-2.pth , skipping...
False
--- Cross validation step [ 3 /  68  ]


...

/tmp/ipykernel_59994/3411885580.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rt_baseline = float(find_runtime(df, kernel, 1, platform))


True
--- Cross validation step [ 67 /  68  ]
--- Training model... 


/tmp/ipykernel_59994/3411885580.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rt_baseline = float(find_runtime(df, kernel, 1, platform))


True


/tmp/ipykernel_59994/3411885580.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rt_baseline = float(find_runtime(df, kernel, 1, platform))


# Results from other works

The accuracies and speedups are taken from the results quoted by NCC in their work for the purpose of comparison. For detailed analysis (discussed later), we run these models and the obtained results are stored as pickle files in ./data/prior_art_results. 

In [32]:
magni_sp_vals = [1.21, 1.01, 0.86, 0.94]
magni_sp_mean = [1.005]
deeptune_sp_vals = [1.10, 1.05, 1.10, 0.99]
deeptune_sp_mean = [1.06]
deeptuneTL_sp_vals = [1.17, 1.23, 1.14, 0.93]
deeptuneTL_sp_mean = [1.1175]
ncc_sp_vals = [1.29, 1.07, 0.97, 1.01]
ncc_sp_mean = [1.086]
inst2vec_sp_vals = [1.228449, 1.252760, 1.109817, 1.154982]
inst2vec_sp_mean = [1.186502]

# IR2Vec Flow-Aware Vs. Others

In [35]:
modernbert_result = pd.DataFrame(
    data,
    columns=[
        "Model",
        "Platform",
        "Kernel",
        "Oracle-CF",
        "Predicted-CF",
        "Speedup",
        "Oracle",
        "OracleSpeedUp",
    ],
)
print("\nSpeedup Matrix: Modernbert Vs. others\n")
modernbert_result_sp_vals = modernbert_result.groupby(["Platform"])["Speedup"].mean().values
modernbert_result_sp_vals_sp_mean = modernbert_result_sp_vals.mean()
sp_df = pd.DataFrame(
    {
        "Magni et al.": magni_sp_vals + magni_sp_mean,
        "DeepTune": deeptune_sp_vals + deeptune_sp_mean,
        "DeepTune-TL": deeptuneTL_sp_vals + deeptuneTL_sp_mean,
        "NCC": ncc_sp_vals + ncc_sp_mean,
        "Inst2Vec": inst2vec_sp_vals + inst2vec_sp_mean,
        'ModernBert': list(modernbert_result_sp_vals) + [modernbert_result_sp_vals_sp_mean],
    },
    index=[
        "AMD Radeon HD 5900",
        "AMD Tahiti 7970",
        "NVIDIA GTX 480",
        "NVIDIA Tesla K20c",
        "Average",
    ],
)
print(sp_df)


Speedup Matrix: Modernbert Vs. others

                    Magni et al.  DeepTune  DeepTune-TL    NCC  Inst2Vec  \
AMD Radeon HD 5900         1.210      1.10       1.1700  1.290  1.228449   
AMD Tahiti 7970            1.010      1.05       1.2300  1.070  1.252760   
NVIDIA GTX 480             0.860      1.10       1.1400  0.970  1.109817   
NVIDIA Tesla K20c          0.940      0.99       0.9300  1.010  1.154982   
Average                    1.005      1.06       1.1175  1.086  1.186502   

                    ModernBert  
AMD Radeon HD 5900    1.438724  
AMD Tahiti 7970       1.379266  
NVIDIA GTX 480        1.169264  
NVIDIA Tesla K20c     1.179483  
Average               1.291684  
